# Notebook to update readme files

In [57]:
import re
import toml
import shutil
import posixpath

from pathlib import Path

import pandas as pd

from mintalib import core


In [58]:
ROOTDIR = Path.cwd().parent
PYPROJECT = ROOTDIR.joinpath("pyproject.toml").resolve(strict=True)


In [50]:
def jquery(data: dict, item: str, default=None):
    result = data

    for i in item.split("."):
        result = result.get(i, None)
        if result is None:
            return default

    return result


def get_project_url(pyproject=PYPROJECT):
    """extract project url from project configuration"""

    if not pyproject.exists():
        raise FileNotFoundError("pyproject.toml")

    config = toml.load(pyproject)
    
    return jquery(config, "project.urls.homepage")

get_project_url()


'https://github.com/furechan/mintalib'

In [51]:
def get_info(func):
    """information about function"""

    info = dict(Name=func.__qualname__)

    doc = func.__doc__ or ""
    description = doc.strip().partition("\n")[0]
    if description is not None:
        info.update(Description=description)

    return info


def list_functions():
    """list functions"""

    result = [v for k, v in vars(core).items() if k.isupper() and callable(v)]

    result = [get_info(f) for f in result]

    result = pd.DataFrame(result).set_index("Name")

    result = result.sort_index()

    return result

In [52]:
functions = list_functions()
functions

,Description
Name,
ADX,Average Directional Index
ATR,Average True Range
AVGPRICE,Average Price
BBANDS,Bollinger Bands
BOP,Balance of Power
CCI,Commodity Channel Index
CMF,Chaikin Money Flow
CROSSOVER,Cross Over
CROSSUNDER,Cross Under


In [53]:
def backup_readme(verbose=True):
    readme = ROOTDIR.joinpath("README.md")
    if ROOTDIR.joinpath("archive").exists():
        backup = ROOTDIR.joinpath("archive/README.bak")
    else:
        backup = ROOTDIR.joinpath("README.bak")

    if verbose:
        print(f"Backing up {readme.name} ...")

    shutil.copy(readme, backup)


backup_readme()

Backing up README.md ...


In [54]:
def update_readme(verbose=True):
    title = "## List of Indicators\n"
    table = list_functions().to_markdown()
    repl = title + "\n" + table + "\n\n\n"

    # flags m=multiline, s=dotall
    pattern = r"(?ms)(^[#]+ List of (Functions|Indicators)\n[^#]+)"

    readme = ROOTDIR.joinpath("README.md")
    contents = readme.read_text()

    output, count = re.subn(pattern, repl, contents)

    if count != 1:
        raise RuntimeError("Cound not locate list of funtions")

    if verbose:
        print(f"Updating {readme.name} ...")

    return readme.write_text(output)


update_readme()

Updating README.md ...


8603

In [55]:
def process_readme(file, *, project_url=None, branch="main", verbose=True):
    """translate relative urls to full urls"""

    if project_url is None:
        project_url = get_project_url()

    def replace(m):
        exclam, alt, url = m.groups()
        ftype = "raw" if exclam else "blob"
        if url.startswith("/"):
            url = url.removeprefix("/")
            url = posixpath.join(project_url, ftype, branch, url)
            text = f"{exclam}[{alt}]({url})"
            if verbose:
                print("mapping", m.group(0), "->", text)
        else:
            text = m.group(0)
        return text

    source = file.read_text()

    result = re.sub(
        r"(\!?) \[ ([^]]*) \] \( ([^)]+) \)", replace, source,
        flags = re.VERBOSE
        )

    return result



mapping [mintalib.core](/docs/mintalib.core.md) -> [mintalib.core](https://github.com/furechan/mintalib/blob/main/docs/mintalib.core.md)
mapping [mintalib.functions](/docs/mintalib.functions.md) -> [mintalib.functions](https://github.com/furechan/mintalib/blob/main/docs/mintalib.functions.md)
mapping [mintalib.indicators](/docs/mintalib.indicators.md) -> [mintalib.indicators](https://github.com/furechan/mintalib/blob/main/docs/mintalib.indicators.md)


In [56]:
README = ROOTDIR.joinpath("README.md").resolve(strict=True)
OUTDIR = ROOTDIR / "build"
OUTDIR.mkdir(exist_ok=True)

output = process_readme(README, verbose=True)

outfile = OUTDIR.joinpath("pypi-readme.md").resolve()

print(f"Updatings {outfile.name} ...")

outfile.write_text(output)

mapping [mintalib.core](/docs/mintalib.core.md) -> [mintalib.core](https://github.com/furechan/mintalib/blob/main/docs/mintalib.core.md)
mapping [mintalib.functions](/docs/mintalib.functions.md) -> [mintalib.functions](https://github.com/furechan/mintalib/blob/main/docs/mintalib.functions.md)
mapping [mintalib.indicators](/docs/mintalib.indicators.md) -> [mintalib.indicators](https://github.com/furechan/mintalib/blob/main/docs/mintalib.indicators.md)
Updatings pypi-readme.md ...


8741